# LangGraph Mini-Lab 3 — Tiny Multi-Tool Agent

**Author:** Dr. Dasha Trofimova

Routes to calculator or Wikipedia tool.

In [ ]:
!pip install -q langgraph wikipedia

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict
import re

def calc_tool(q: str) -> str:
    if not re.fullmatch(r"[\d\s+\-*/().]+", q):
        return "Calculator supports only + - * / and numbers."
    try:
        return str(eval(q, {"__builtins__": {}}, {}))
    except Exception as e:
        return f"Calc error: {e}"

def wiki_tool(q: str) -> str:
    import wikipedia
    wikipedia.set_lang("en")
    try:
        page = wikipedia.page(q, auto_suggest=True, redirect=True)
        return page.title + ": " + wikipedia.summary(page.title, sentences=3)
    except Exception as e:
        try:
            hits = wikipedia.search(q)
            if hits:
                return wikipedia.summary(hits[0], sentences=3)
        except Exception:
            pass
        return f"Wikipedia error: {e}"

class AState(TypedDict):
    query: str
    answer: str
    route: str

def decide(state: AState) -> AState:
    q = state["query"].strip()
    if re.search(r"[\d][\d\s+\-*/().]*", q):
        return {"route": "calc", "answer": ""}
    if q.lower().startswith("wiki:"):
        return {"route": "wiki", "answer": ""}
    return {"route": "echo", "answer": ""}

def run_calc(state: AState) -> AState:
    return {"answer": calc_tool(state["query"])}

def run_wiki(state: AState) -> AState:
    q = state["query"].replace("wiki:", "").strip()
    return {"answer": wiki_tool(q)}

def run_echo(state: AState) -> AState:
    return {"answer": f"(echo) {state['query']}"}

def router(state: AState):
    return state["route"]

g = StateGraph(AState)
g.add_node("decide", decide)
g.add_node("calc", run_calc)
g.add_node("wiki", run_wiki)
g.add_node("echo", run_echo)
g.add_conditional_edges("decide", router, {"calc": "calc", "wiki": "wiki", "echo": "echo"})
g.add_edge("calc", END); g.add_edge("wiki", END); g.add_edge("echo", END)
g.set_entry_point("decide")
agent = g.compile()

print(agent.invoke({"query": "2*(3+4)", "answer": "", "route": ""}))
print(agent.invoke({"query": "wiki: Berlin", "answer": "", "route": ""}))
print(agent.invoke({"query": "hello there", "answer": "", "route": ""}))

### 🧭 Graph Visualization
Router decides which tool to run based on the query.

In [ ]:
!apt-get -qq update && apt-get -qq install -y graphviz > /dev/null
!pip install -q graphviz

In [ ]:
from graphviz import Digraph
gdot = Digraph(comment="Multi-Tool Agent", format="png")
gdot.attr(rankdir="LR", bgcolor="white")
gdot.node("decide", "decide()", shape="diamond", style="rounded,filled", fillcolor="#F3F4F6")
gdot.node("calc", "run_calc()", shape="box", style="rounded,filled", fillcolor="#DBEAFE")
gdot.node("wiki", "run_wiki()", shape="box", style="rounded,filled", fillcolor="#E9D5FF")
gdot.node("echo", "run_echo()", shape="box", style="rounded,filled", fillcolor="#DCFCE7")
gdot.edge("decide", "calc", label="looks like math")
gdot.edge("decide", "wiki", label="prefix: 'wiki:'")
gdot.edge("decide", "echo", label="otherwise")
display(gdot)

Examples:
- `2*(3+4)` → **calc**
- `wiki: Berlin` → **wiki**
- `hello there` → **echo**


## 🧭 Agent Anatomy — Tiny Multi‑Tool Agent Graph

**What this agent does:**  
Decides which tool to run based on the `query`:
- **calc** → arithmetic expression with digits/operators
- **wiki** → queries prefixed with `wiki:` (uses `wikipedia` package)
- **echo** → fallback

**State schema:**  
- `query` *(str)* → user input  
- `route` *(str)* → routing tag chosen by `decide` (`calc|wiki|echo`)  
- `answer` *(str)* → final answer

**Nodes:**  
- **decide** → inspects `query` and sets `route`.  
- **calc** → runs a safe calculator.  
- **wiki** → fetches a short summary.  
- **echo** → echoes input.  
Routing is handled by a conditional edge using `route`.


In [ ]:

from langgraph.graph import StateGraph, END
from typing import TypedDict
import re

class AState(TypedDict):
    query: str
    answer: str
    route: str  # 'calc' | 'wiki' | 'echo'

node_desc = {
    "decide": "Router: sets route='calc' for math, 'wiki' for 'wiki:' prefix, else 'echo'.",
    "calc":   "Calculator tool: evaluates + - * / expressions safely.",
    "wiki":   "Wikipedia tool: returns a short summary for a topic.",
    "echo":   "Echo tool: returns the input unchanged with a prefix."
}

def trace(name):
    def deco(fn):
        def wrapped(state: AState):
            print(f"\n▶ Node: {name} — {node_desc[name]}")
            out = fn(state)
            print(f"   Input: {state}")
            print(f"   Output: {out}")
            return out
        wrapped.__doc__ = node_desc[name]
        return wrapped
    return deco

def calc_tool(q: str) -> str:
    if not re.fullmatch(r"[\\d\\s+\\-*/().]+", q):
        return "Calculator supports only + - * / and numbers."
    try:
        return str(eval(q, {"__builtins__": {}}, {}))
    except Exception as e:
        return f"Calc error: {e}"

def wiki_tool(q: str) -> str:
    import wikipedia
    wikipedia.set_lang("en")
    try:
        page = wikipedia.page(q, auto_suggest=True, redirect=True)
        return page.title + ": " + wikipedia.summary(page.title, sentences=3)
    except Exception as e:
        try:
            hits = wikipedia.search(q)
            if hits:
                return wikipedia.summary(hits[0], sentences=3)
        except Exception:
            pass
        return f"Wikipedia error: {e}"

@trace("decide")
def decide(state: AState) -> AState:
    q = state["query"].strip()
    if re.search(r"[\\d][\\d\\s+\\-*/().]*", q):
        return {"route": "calc", "answer": ""}
    if q.lower().startswith("wiki:"):
        return {"route": "wiki", "answer": ""}
    return {"route": "echo", "answer": ""}

@trace("calc")
def run_calc(state: AState) -> AState:
    return {"answer": calc_tool(state["query"]), "route": state["route"]}

@trace("wiki")
def run_wiki(state: AState) -> AState:
    q = state["query"].replace("wiki:", "").strip()
    return {"answer": wiki_tool(q), "route": state["route"]}

@trace("echo")
def run_echo(state: AState) -> AState:
    return {"answer": f\"(echo) {state['query']}\", "route": state["route"]}

def router(state: AState):
    return state["route"]

g = StateGraph(AState)
g.add_node("decide", decide)
g.add_node("calc", run_calc)
g.add_node("wiki", run_wiki)
g.add_node("echo", run_echo)
g.add_conditional_edges("decide", router, {"calc": "calc", "wiki": "wiki", "echo": "echo"})
g.add_edge("calc", END); g.add_edge("wiki", END); g.add_edge("echo", END)
g.set_entry_point("decide")

agent = g.compile()

print("\n=== Trace: calc ===")
print(agent.invoke({"query": "2*(3+4)", "answer": "", "route": ""}))
print("\n=== Trace: wiki ===")
print(agent.invoke({"query": "wiki: Berlin", "answer": "", "route": ""}))
print("\n=== Trace: echo ===")
print(agent.invoke({"query": "hello there", "answer": "", "route": ""}))
